In [163]:
%run io_functions.ipynb

In [123]:
all_files2 = ['part/' + p for p in os.listdir('part/') if p.startswith('timestamp')]
all_files3 = ['tss_null/' + p for p in os.listdir('tss_null/') if p.startswith('timestamp')]
all_files4 = ['part_null/' + p for p in os.listdir('part_null/') if p.startswith('timestamp')]

In [124]:
read_and_store('part1.csv', 'part_null', timestampColumnName='time', global_granularity='1 day')

In [133]:
df = unpack('part_null', all_files4)

In [134]:
df.show(100)

+-----+----+-----+----+----+--------+--------------------+--------------------+----------+
|PM2_5|PM10|PM1_0| NO2|  BC|activity|               event|         coordinates| timestamp|
+-----+----+-----+----+----+--------+--------------------+--------------------+----------+
|   12|  13|   13|NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.5...|1560314880|
|    4|   4|    4|NULL|1105|     Rue|                NULL|[48.83594167, 2.5...|1560317520|
|    1|   1|    1|NULL| 917|     Bus|                NULL| [48.83648, 2.52973]|1560320160|
|    1|   1|    1|NULL| 974|     Rue|                NULL|[48.83635833, 2.5...|1560320580|
|    2|   2|    2|NULL| 846|  Bureau|Ouverture De Fenêtre|[48.83620333, 2.5...|1560320880|
|    0|   0|    0|NULL| 114|     Rue|                NULL|[48.82939833, 2.3...|1560353880|
|   18|  20|   20|NULL|2198|   Métro|                NULL|[48.83160167, 2.3...|1560355560|
|    0|   0|    0|NULL| 364|Domicile|                NULL|[48.826745, 2.355...|1560358680|

In [156]:
%run operators.ipynb

In [151]:
df1 = temporal_projection('TProj[PM10*2]', df)

In [152]:
df1.show()

+-----+-----+-----+----+----+--------+--------------------+--------------------+----------+
|PM2_5| PM10|PM1_0| NO2|  BC|activity|               event|         coordinates| timestamp|
+-----+-----+-----+----+----+--------+--------------------+--------------------+----------+
|   12| 26.0|   13|NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.5...|1560314880|
|    4|  8.0|    4|NULL|1105|     Rue|                NULL|[48.83594167, 2.5...|1560317520|
|    1|  2.0|    1|NULL| 917|     Bus|                NULL| [48.83648, 2.52973]|1560320160|
|    1|  2.0|    1|NULL| 974|     Rue|                NULL|[48.83635833, 2.5...|1560320580|
|    2|  4.0|    2|NULL| 846|  Bureau|Ouverture De Fenêtre|[48.83620333, 2.5...|1560320880|
|    0|  0.0|    0|NULL| 114|     Rue|                NULL|[48.82939833, 2.3...|1560353880|
|   18| 40.0|   20|NULL|2198|   Métro|                NULL|[48.83160167, 2.3...|1560355560|
|    0|  0.0|    0|NULL| 364|Domicile|                NULL|[48.826745, 2.355...|

In [157]:
df1 = temporal_aggregation('TAgg[1 day,avg(PM10)]', df)

In [158]:
df1.show()

+-----+------------------+
|group|         avg(PM10)|
+-----+------------------+
|    0| 40.61538461538461|
|    1|3.3333333333333335|
|    3|1.6666666666666667|
|    4|              14.0|
|    5|2.3333333333333335|
+-----+------------------+



In [161]:
df1 = window_aggregation('WAgg[1 day,avg(PM10)]', df)

In [162]:
df1.show()

+-----+----+-----+----+----+--------+--------------------+--------------------+----------+------------------+
|PM2_5|PM10|PM1_0| NO2|  BC|activity|               event|         coordinates| timestamp|         avg(PM10)|
+-----+----+-----+----+----+--------+--------------------+--------------------+----------+------------------+
|   12|  13|   13|NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.5...|1560314880| 40.61538461538461|
|    4|   4|    4|NULL|1105|     Rue|                NULL|[48.83594167, 2.5...|1560317520|42.916666666666664|
|    1|   1|    1|NULL| 917|     Bus|                NULL| [48.83648, 2.52973]|1560320160| 46.45454545454545|
|    1|   1|    1|NULL| 974|     Rue|                NULL|[48.83635833, 2.5...|1560320580|              51.0|
|    2|   2|    2|NULL| 846|  Bureau|Ouverture De Fenêtre|[48.83620333, 2.5...|1560320880| 56.55555555555556|
|    0|   0|    0|NULL| 114|     Rue|                NULL|[48.82939833, 2.3...|1560353880|              51.5|
|   18|  2

In [84]:
df = spark.read.csv('part1.csv', inferSchema=False, header=True, timestampFormat='yyyy-MM-dd HH:mm:ss+ss')
df = df.withColumnRenamed('time', 'timestamp')
df = df.withColumn('timestamp', to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss+ss'))      
    # Checking and changing column names because Spark does not take columns with dot in them
df = df.toDF(*change_column_names(df.columns))

    
    # Change the timestamp to epoch
df = df.withColumn("timestamp_epoch", df.timestamp.cast("long")).sort('timestamp_epoch')

    # Using the concat_udf to merge together the lat and lon
concat_udf = udf(concat, StringType())
    
cols = [c for c in df.columns if not c.startswith("timestamp") and not c.startswith(
    "lat") and not c.startswith("lon")]
coords = [c for c in df.columns if c.startswith("lat") or c.startswith("lon")]
df1 = df.select('timestamp_epoch', *cols, concat_udf(*coords).alias('coordinates'))

In [85]:
df1 = df1.drop_duplicates(['timestamp_epoch']).sort('timestamp_epoch')
df1.show()  

+---------------+-----+----+-----+----+----+--------+--------------------+--------------------+
|timestamp_epoch|PM2_5|PM10|PM1_0| NO2|  BC|activity|               event|         coordinates|
+---------------+-----+----+-----+----+----+--------+--------------------+--------------------+
|     1560314880|   12|  13|   13|NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.5...|
|     1560317520|    4|   4|    4|NULL|1105|     Rue|                NULL|[48.83594167, 2.5...|
|     1560320160|    1|   1|    1|NULL| 917|     Bus|                NULL| [48.83648, 2.52973]|
|     1560320580|    1|   1|    1|NULL| 974|     Rue|                NULL|[48.83635833, 2.5...|
|     1560320880|    2|   2|    2|NULL| 846|  Bureau|Ouverture De Fenêtre|[48.83620333, 2.5...|
|     1560353880|    0|   0|    0|NULL| 114|     Rue|                NULL|[48.82939833, 2.3...|
|     1560355560|   18|  20|   20|NULL|2198|   Métro|                NULL|[48.83160167, 2.3...|
|     1560358680|    0|   0|    0|NULL| 

In [86]:
    # Getting the inner granularity in seconds
step = calculate_step_from_string('1 minute')
    
    
minp, maxp = df1.select(min("timestamp_epoch"), max("timestamp_epoch")).first()
print(minp)
reference = spark.range(
    (minp / step) * step, ((maxp / step) + 1) * step, step).select(
    col("id").cast("timestamp").cast("long").alias("timestamp"))
        
            # Fill the skipped values with nulls
df_joined = reference.join(df1,reference.timestamp == df1.timestamp_epoch, "leftouter").drop(
    'timestamp_epoch')
df_joined = df_joined.na.fill('null')
        
 

1560314880


In [62]:
df_joined.show()

+----------+-----+----+-----+----+----+--------+--------------------+--------------------+
| timestamp|PM2_5|PM10|PM1_0| NO2|  BC|activity|               event|         coordinates|
+----------+-----+----+-----+----+----+--------+--------------------+--------------------+
|1560314880|   12|  13|   13|NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.5...|
|1560314940| null|null| null|null|null|    null|                null|                null|
|1560315000| null|null| null|null|null|    null|                null|                null|
|1560315060| null|null| null|null|null|    null|                null|                null|
|1560315120| null|null| null|null|null|    null|                null|                null|
|1560315180| null|null| null|null|null|    null|                null|                null|
|1560315240| null|null| null|null|null|    null|                null|                null|
|1560315300| null|null| null|null|null|    null|                null|                null|

In [87]:
df_joined.agg(collect_list('BC')).show(1, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
   # Adding the global timestamp to group by it
global_step = calculate_step_from_string('1 day')
tsGroup = (floor(df_joined.timestamp / lit(global_step)) * lit(global_step)).alias('timestamp_interval')
        
    # A lambda to get the first value from an array
getFirst = udf(lambda x:x[0], StringType())
    
    # Collect the different observations in lists, in order to store them
columns = cols + ['coordinates']
a = [collect_list(c).alias(c) for c in columns]
        
    # Create the dataframe to save when all observations are packed depending on the global granularity
df2 = (df_joined.groupBy(tsGroup).agg(
*a, collect_list('timestamp').alias('timestamp_first'))).withColumn(
        'timestamp_first', getFirst('timestamp_first')).orderBy('timestamp_first')
    
    # Add the granularities meta-data
df2 = df2.withColumn('global_granularity', lit(global_granularity))
df2 = df2.withColumn('inner_granularity', lit(inner_granularity))

In [47]:
df = unpack('part', all_files2)

In [36]:
df = unpack('tss_null', all_files3)

In [48]:
df.show(100)

+-----+----+-----+----+----+--------+--------------------+--------------------+----------+
|PM2_5|PM10|PM1_0| NO2|  BC|activity|               event|         coordinates| timestamp|
+-----+----+-----+----+----+--------+--------------------+--------------------+----------+
|   12|  13|   13|NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.5...|1560314880|
|    4|   4|    4|NULL|1105|     Rue|                NULL|[48.83594167, 2.5...|1560314940|
|    1|   1|    1|NULL| 917|     Bus|                NULL| [48.83648, 2.52973]|1560315000|
|    1|   1|    1|NULL| 974|     Rue|                NULL|[48.83635833, 2.5...|1560315060|
|    2|   2|    2|NULL| 846|  Bureau|Ouverture De Fenêtre|[48.83620333, 2.5...|1560315120|
|    0|   0|    0|NULL| 114|     Rue|                NULL|[48.82939833, 2.3...|1560315180|
|   18|  20|   20|NULL|2198|   Métro|                NULL|[48.83160167, 2.3...|1560315240|
|    0|   0|    0|NULL| 364|Domicile|                NULL|[48.826745, 2.355...|1560315300|

In [49]:
df.count()

33